<a href="https://colab.research.google.com/github/MOUNYASREE/sithafal-Tasks/blob/main/TASK2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required libraries
!pip install requests transformers faiss-cpu

import requests
from bs4 import BeautifulSoup
from transformers import AutoTokenizer, AutoModel
import torch
import faiss
import numpy as np

# Load Hugging Face model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

# Step 1: Crawl and Scrape
def crawl_and_scrape(urls):
    website_data = {}
    for url in urls:
        try:
            response = requests.get(url)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, "html.parser")
            text = " ".join([p.get_text() for p in soup.find_all("p")])
            website_data[url] = text
        except Exception as e:
            print(f"Error scraping {url}: {e}")
    return website_data


In [ ]:
# Step 2: Chunk and Embed
def chunk_and_embed(data, tokenizer, model, chunk_size=300):
    chunks = []
    embeddings = []
    for url, content in data.items():
        words = content.split()
        for i in range(0, len(words), chunk_size):
            chunk = " ".join(words[i:i + chunk_size])
            if len(chunk.strip()) > 0:
                chunks.append((url, chunk.strip()))
                tokens = tokenizer(chunk, return_tensors="pt", truncation=True, padding=True)
                with torch.no_grad():
                    embedding = model(**tokens).pooler_output.squeeze().numpy()
                    embeddings.append(embedding)
    return chunks, np.array(embeddings)

In [ ]:

# Step 3: Store in FAISS
def store_embeddings(embeddings):
    d = embeddings.shape[1]  # Dimension of embeddings
    index = faiss.IndexFlatL2(d)
    index.add(embeddings)
    return index

In [ ]:
# Step 4: Query Handling
def query_vector_database(query, index, tokenizer, model, chunks, top_k=5):
    tokens = tokenizer(query, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        query_embedding = model(**tokens).pooler_output.numpy()
    distances, indices = index.search(query_embedding, k=top_k)
    results = [chunks[i] for i in indices[0] if i < len(chunks)]
    return results

In [ ]:

# Step 5: Generate Response
def generate_response(results):
    if not results:
        return "Sorry, I couldn't find any relevant information."
    response = "Based on your query, here are the results:\n\n"
    for url, text in results:
        response += f"URL: {url}\nContent: {text[:200]}...\n\n"
    return response

In [ ]:
# Main Function
def main():
    # Step 1: Define URLs
    urls = [
        "https://www.uchicago.edu/",
        "https://www.washington.edu/",
        "https://www.stanford.edu/",
        "https://und.edu/",
    ]

    # Step 2: Crawl and scrape websites
    print("Crawling and scraping websites...")
    website_data = crawl_and_scrape(urls)

    # Step 3: Chunk and embed content
    print("Chunking and embedding content...")
    chunks, embeddings = chunk_and_embed(website_data, tokenizer, model)

    # Step 4: Store embeddings in FAISS
    print("Storing embeddings in FAISS...")
    index = store_embeddings(embeddings)

    # Step 5: Handle user queries
    while True:
        query = input("\nEnter your query (or 'exit' to quit): ")
        if query.lower() == "exit":
            print("Exiting the program.")
            break
        print("Searching content...")
        results = query_vector_database(query, index, tokenizer, model, chunks)
        print("Generating response...")
        response = generate_response(results)
        print(response)

if __name__ == "__main__":
    main()

Crawling and scraping websites...
Error scraping https://www.uchicago.edu/: 403 Client Error: Forbidden for url: https://www.uchicago.edu/
Chunking and embedding content...
Storing embeddings in FAISS...

Enter your query (or 'exit' to quit): "research opportunities at the University of Chicago"?
Searching content...
Generating response...
Based on your query, here are the results:

URL: https://www.stanford.edu/
Content: Other ways to search: Map Profiles Stanford Explore Stanford Stanford was founded almost 150 years ago on a bedrock of societal purpose. Our mission is to contribute to the world by educating students...

URL: https://www.washington.edu/
Content: UW astronomy undergrads are using cutting-edge coding skills to help scientists make the most of discoveries from a revolutionary new telescope. Read story Chris Mantegna, â21, is studying how pollu...

URL: https://www.stanford.edu/
Content: a vibrant community of creative and accomplished people from around the world A re